In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
spark = SparkSession.builder.appName("movieRecommendationPySpark").getOrCreate()

In [3]:
ratings = (
    spark.read.csv(
        path = "../data/ml-25m/ratings.csv",
        sep=",", header=True,quote='"',schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
    ).select("userId", "movieId", "rating")
    .cache()
)

AnalysisException: Path does not exist: file:/e:/Học/Năm 4/Công nghệ dữ liệu lớn/Demo/data/ml-25/ratings.csv

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
als = ALS(
    userCol ="userId",
    itemCol ="movieId",
    ratingCol = "rating",
)
(training_data, validation_data) = ratings.randomSplit([8.0,2.0])

evaluator = RegressionEvaluator(
    metricName="rmse",labelCol="rating",predictionCol="prediction"
)

model = als.fit(training_data)

prediction = model.transform(validation_data)



In [ ]:
# prediction.show(10,False)
rmse = evaluator.evaluate(prediction.na.drop())
# print(rmse)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|1     |673    |3.0   |3.286532  |
|1     |1208   |4.0   |5.110944  |
|1     |1      |4.0   |4.5577874 |
|1     |3      |4.0   |4.065167  |
|1     |553    |5.0   |4.3488336 |
|1     |1256   |5.0   |4.9037333 |
|1     |480    |4.0   |4.363128  |
|1     |608    |5.0   |4.887341  |
|1     |1024   |5.0   |3.968002  |
|1     |1090   |4.0   |4.827309  |
+------+-------+------+----------+
only showing top 10 rows

0.8751367980191345


In [ ]:
# userFactors=model.userFactors
# itemFactors = model.itemFactors
# userFactors.sort('id').show(5,False)
# itemFactors.sort('id').show(5,False)
# import numpy as np
# user91Features = model.userFactors.filter(f.col('id')==91).select(f.col('features')).rdd.flatMap(lambda x:x).collect()[0]
# item471Features = model.itemFactors.filter(f.col('id')==471).select(f.col('features')).rdd.flatMap(lambda x:x).collect()[0]

# print(user91Features)
# print(item471Features)
# print('Predicted rating of user 91 for movie 471: ' + str(np.dot(user91Features, item471Features)))

D:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+---+----------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                    |
+---+----------------------------------------------------------------------------------------------------------------------------+
|1  |[0.2910618, -0.12626813, 0.53618056, 0.34501067, -0.1196083, 0.013863539, 0.21544614, 2.0078578, 0.66096014, -0.018578209]  |
|2  |[0.79601717, 0.2603853, -0.117772736, 0.68528473, 0.30389938, 0.057525698, 0.067037456, 1.6126558, 0.39602172, -0.28178793] |
|3  |[-0.77848583, 0.7634151, -0.583678, 0.5371052, 0.7707833, 0.18535544, 0.62902284, 0.4933478, 0.7852314, 1.1517444]          |
|4  |[0.64204293, -0.88842577, 0.5375029, -0.3971977, -0.30614096, -0.23317055, -0.06744804, 1.6801956, 0.39346108, 0.680426]    |
|5  |[-0.0010875319, -0.3232852, 0.55465907, -0.2005563, -0.70407206, 0.21196328, 0

In [ ]:
# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# param_grid =  (
#     ParamGridBuilder().addGrid(als.rank,[5,10]).addGrid(als.maxIter,[20]).addGrid(als.regParam,[0.05,1]).build()
# )
# print(param_grid)

[{Param(parent='ALS_74c46e483e0a', name='rank', doc='rank of the factorization'): 5, Param(parent='ALS_74c46e483e0a', name='maxIter', doc='max number of iterations (>= 0).'): 20, Param(parent='ALS_74c46e483e0a', name='regParam', doc='regularization parameter (>= 0).'): 0.05}, {Param(parent='ALS_74c46e483e0a', name='rank', doc='rank of the factorization'): 5, Param(parent='ALS_74c46e483e0a', name='maxIter', doc='max number of iterations (>= 0).'): 20, Param(parent='ALS_74c46e483e0a', name='regParam', doc='regularization parameter (>= 0).'): 1.0}, {Param(parent='ALS_74c46e483e0a', name='rank', doc='rank of the factorization'): 10, Param(parent='ALS_74c46e483e0a', name='maxIter', doc='max number of iterations (>= 0).'): 20, Param(parent='ALS_74c46e483e0a', name='regParam', doc='regularization parameter (>= 0).'): 0.05}, {Param(parent='ALS_74c46e483e0a', name='rank', doc='rank of the factorization'): 10, Param(parent='ALS_74c46e483e0a', name='maxIter', doc='max number of iterations (>= 0).

In [ ]:
# rmse = evaluator.evaluate(prediction.na.drop())
# print(rmse)

0.8751367980191345


In [ ]:
# model.userFactors.show(5,False)

+---+-------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                 |
+---+-------------------------------------------------------------------------------------------------------------------------+
|10 |[0.234548, -0.31415012, 0.053325303, -0.7197765, 0.011542611, -0.9042637, -0.44329885, 1.1098106, 1.0720254, -0.85817134]|
|20 |[0.5004493, -0.18024325, 0.08042228, -0.4120407, -0.82010114, 0.24364965, -0.32089552, 1.5661156, 1.0555433, -0.18861222]|
|30 |[0.29699945, 0.5122262, 0.7743655, 0.26239362, -0.23122218, -0.22499156, 0.2205979, 1.5108275, 1.0886868, -0.24839555]   |
|40 |[0.962316, 0.24725015, 0.2506125, 0.18621314, -0.3400564, 0.069881424, 0.31563887, 1.7079871, 0.6658317, 0.10538066]     |
|50 |[0.11444859, 0.07726337, -0.04935533, -0.12985186, -0.479587, 0.25315705, 0.28283778, 1.2838686, 0.

In [ ]:
# prediction.toPandas()

,userId,movieId,rating,prediction
0,593,1580,1.5,3.060124
1,597,1580,3.0,3.781061
2,34,1580,2.5,3.102221
3,368,2366,4.0,3.169171
4,115,1580,4.0,3.457590
...,...,...,...,...
20082,535,2706,4.0,1.946153
20083,420,48385,3.5,3.572902
20084,18,2706,3.5,3.319682
20085,483,66785,4.0,3.423220


In [ ]:
rec_all_users = model.recommendForAllUsers(5).cache()
# rec_all_users.show(5,False)
# rec_all_users.printSchema()

+------+-------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                  |
+------+-------------------------------------------------------------------------------------------------+
|471   |[{3022, 4.9927526}, {89904, 4.8837285}, {446, 4.6952806}, {1446, 4.609901}, {106100, 4.603286}]  |
|463   |[{78836, 4.6696997}, {69524, 4.659609}, {86377, 4.625015}, {3022, 4.5858045}, {665, 4.581}]      |
|496   |[{213, 5.9084764}, {70994, 5.41703}, {3266, 4.9417677}, {6818, 4.92134}, {8973, 4.9139323}]      |
|148   |[{2843, 4.887899}, {3404, 4.6965885}, {26073, 4.5484962}, {7071, 4.5484962}, {117531, 4.5484962}]|
|540   |[{78836, 5.2324185}, {171495, 5.161601}, {117531, 5.133931}, {7071, 5.133931}, {26073, 5.133931}]|
+------+-------------------------------------------------------------------------------------------------+
only showing top 5 rows

root
 |-- us

In [ ]:
# recommendations_for_user91 = rec_all_users.filter(f.col('userId')==110)
# movies = (
#     spark.read.csv(
#         path = "./data/ml-25/movies.csv",
#         sep=",",
#         header = True,
#         quote='"',
#         schema = "movieId INT, title STRING, genres STRING",
#     )
# )
# movieid = recommendations_for_user91.withColumn('movie',f.explode('recommendations')).withColumn('movie',f.col('movie.movieId')).select('movie')
# movie_for_user91= movies.join(movieid,movies.movieId==movieid.movie,'inner').withColumn("title_year",f.split(f.col("title"),"\(")).withColumn("year",f.substring(f.col("title_year").getItem(1),0,4)).withColumn("titlestring",f.col("title_year").getItem(0)).select('movieId','titlestring','year')
# movie_for_user91.show(5,False)

+-------+------------------------------------------+----+
|movieId|titlestring                               |year|
+-------+------------------------------------------+----+
|3022   |General, The                              |1926|
|3379   |On the Beach                              |1959|
|7096   |Rivers and Tides                          |2001|
|89904  |The Artist                                |2011|
|177593 |Three Billboards Outside Ebbing, Missouri |2017|
+-------+------------------------------------------+----+



In [ ]:
ratedMovies = ratings.filter(f.col('userId')==110).select('movieId').rdd.flatMap(lambda x:x).collect()

movies_to_be_rated = (
    ratings.filter(~ f.col('movieId').isin(ratedMovies))
    .select('movieId').distinct().withColumn('userId',f.lit(110))
)
# movies_to_be_rated.sort('movieId').show(5)

user_movie_predictions = model.transform(movies_to_be_rated)
user_movie_predictions.filter(~f.isnan('prediction')).orderBy('prediction',ascending=False).show(5)

+-------+------+
|movieId|userId|
+-------+------+
|      1|   110|
|      2|   110|
|      3|   110|
|      4|   110|
|      5|   110|
+-------+------+
only showing top 5 rows

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|  89904|   110|  4.816396|
|   3022|   110| 4.7419705|
| 177593|   110| 4.6890345|
|   3379|   110|  4.661535|
|   1212|   110|  4.648019|
+-------+------+----------+
only showing top 5 rows

